## MACHINE LEARNING WITH $k$ NEAREST NEIGHBOUR ALGORITHM

>by Dr Juan H Klopper

- Research Fellow
- School for Data Science and Computational Thinking
- Stellenbosch University

## INTRODUCTION

The $k$ nearest neighbour (kNN) algorithm is one of the easiest machine learning algorithms to understand and implement. It can be used for classification and regression problems. In the former, the target variable is categorical. In the latter, it is numerical.

In this notebook we explore the $k$ nearest neighbour machine learning (ML) algorithm. We start of with a simple example of classification before embarking on solving a more realistic problem. Along the way we will learn a lot of the basic concepts of ML. We end with a small example to help us understand how to use kNN in a regression problem.

To note upfront, for the same objects, ML uses different names than we use in statistics. Independent variables are referred to as __feature variables__ or simply __features__. The dependent variable is termed a __target variable__ or an __outcome variable__ (or simply a __target__ or an __outcome__. If the target variable is categorical, then the sample space elements are termed __classes__.

## PACKAGES USED IN THIS NOTEBOOK

The following packages will be used in this notebook.

In [ ]:
import numpy as np
from pandas import DataFrame, Series, read_csv

In [ ]:
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_white'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%config InlineBackend.figure_format = "retina" # For Retina type displays

In [ ]:
# Format tables printed to the screen (don't put this on the same line as the code)
%load_ext google.colab.data_table

In [ ]:
from google.colab import drive  # Connect to Google Drive

## THE NEAREST NEIGHBOURS CONCEPT

### $k$ NEAREST NEIGHBOUR CLASSIFICATION

The $k$ nearest neighbour classifier classifies an observation based on the classes in its vicinity. Vicinity infers distance. With this ML algorithm, we measure a distance between observations.

There are various ways to define distance. Euclidean distance (a straight line on a flat surface) is most familiar to us. We consider a single numerical variable (for a featuare variable) and two classes (for a binary target variable). The code below generates a pandas dataframe object, with two appropriately named variables.

In [ ]:
np.random.seed(42)

df = DataFrame(
    {'Feature':np.random.randint(10, high=20, size=7),
     'Target':np.random.choice(['A', 'B'], size=7)}
)
df

With the random seed set as $42$, we note four observations belonging to group A and three to group B.

A scatter plot can be used to visualise this single variable for two classes. Note that two observations in group A have a feature variable value of $16$.

In [ ]:
single_dim_fig = go.Figure(
    go.Scatter(
        x=df.loc[df.Target == 'A'].Feature,
        y=[0, 0, 0, 0],
        name='A',
        mode='markers',
        marker={'size':20}
    )
).add_trace(
    go.Scatter(
        x=df.loc[df.Target == 'B'].Feature,
        y=[0, 0, 0],
        name='B',
        mode='markers',
        marker={'size':20}
    )
).update_layout(
    title='Variable values for two classes',
    xaxis={'title':'Variable value'}
)

single_dim_fig

Distance, $d$, between any two values, $x_{1}$ and $x_{2}$, for this single dimension (all values are on a single axis) is given in (1). Distance is always a positive value, hence we take the absolute value of the difference for two points $x_{1}$ and $x_{2}$.

$$d = \lvert x_{1} - x_{2} \rvert \tag{}$$

The distance between $12$ and $19$ is therefor $\lvert 12 - 19 \rvert = \lvert -7 \rvert = 7$.

Now we introduce an unknown observation with a value of $12.5$. The $k$ in $k$ nearest neigbours is an integer (whole number) reflecting the number of neighbours to an observation. It is set at an odd value. In this instance, we shall say $k=3$.

In [ ]:
single_dim_fig.add_trace(
    go.Scatter(
        x=[12.5],
        y=[0],
        name='Unkown class',
        mode='markers',
        marker={'size':20}
    )
)

single_dim_fig

The three nearest neigbours are $12, 13$, and $14$. The distance to these nearest (closets) neighbours are $\lvert 12.5 - 12 \rvert = 0.5$, $\lvert 12.5 - 13 \rvert = 0.5$, and $\lvert 12.5 - 14 \rvert = 1.5$.

We note that the classes for these three nearest neigbours are group B, group B, and group A. Since we chose an odd number of neighbors, we can simply take a _majority vote_. This would be group B. The $k$ nearest neighbour classifier would therefor classify this new observation as belonging to target class `B`.

If we add another numerical variable, we can plot the data as a scatter plot in the plane. We do this below after adding another random set of values.

In [ ]:
np.random.seed(42)

df['Feature2'] = np.random.randint(100, 200, 7) / 10

In [ ]:
df

In [ ]:
two_dim_fig = go.Figure(
    go.Scatter(
        x=df.loc[df.Target == 'A'].Feature,
        y=df.loc[df.Target == 'A'].Feature2,
        name='Group A',
        mode='markers',
        marker={'size':20}
    )
).add_trace(
    go.Scatter(
        x=df.loc[df.Target == 'B'].Feature,
        y=df.loc[df.Target == 'B'].Feature2,
        name='Group B',
        mode='markers',
        marker={'size':20}
    )
).update_layout(
    title='Variable values for two classes',
    xaxis={'title':'Feature value'},
    yaxis={'title':'Feature 2 value'}
)

two_dim_fig

The equation for the Euclidean distance in the plane (two dimensional space) is the Pythagorean Theorem and is shown in (2) for two points in the plane, $P_{1} = \left( x_{1} , y_{1} \right)$ and $P_{2} = \left( x_{2} , y_{2} \right)$.

$$d \left( P_{1} , P_{2} \right) = \sqrt{{\left( x_{1} - x_{2} \right)}^{2} + {\left( y_{1} - y_{2}  \right)}^{2}} \tag{2}$$

We are only interested in the positive value of the square root. Since both expressions in the square root are squared, we will always have a value of greater than or equal to $0$ and can therefor take the square root.

A new observation with values `Feature` $=14$ and `Feature2` $=18$ is shown below.

In [ ]:
two_dim_fig.add_trace(
    go.Scatter(
        x=[14],
        y=[18],
        name='Unkown class',
        mode='markers',
        marker={'size':20}
    )
).update_yaxes(
    scaleanchor='x',
    scaleratio=1,
  ) # For same x and y axis scale

two_dim_fig

The nearest $k=3$ neigbours are group A, group B, and group B. This unknown observation is therefor classified as belonging to group B. Below, we set up a function to calculate this distance and use it for the three nearest neighbours.

In [ ]:
def dist_2D(x1, y1, x2, y2):
  distance = np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
  return distance

In [ ]:
dist_2D(14, 18, 14, 17.1) # Group B observation

In [ ]:
dist_2D(14, 18, 13, 19.2) # Group A observation

In [ ]:
dist_2D(14, 18, 12, 18.2) # Group B observation

All other observations are _further_ away.

The scikit-learn package has many ML algorithms including a $k$ nearest neighbour classifier (for classification problems). We will use this classifier in an example. First, though, we will use the `make_classification` function from the datasets module of scikit-learn. This function generates random value datasets for ML tasks. The code comment explains the arguments used. The documentation for this function list all the other arguments, which we will leave at their default values.

In [ ]:
X, y = make_classification(
    n_samples=200, # Number of observations
    n_features=5, # Number of features
    n_informative=3, # Number of features that are informative as to the class
    n_redundant=2, # Number of redundant feautres
    n_classes=2, # Setting a binary target variable
    flip_y=0.1, # Flip 10% of the observations to the other class
    random_state=42 # Seeding the pseudo-random number generator
)

The function returns two arrays, which we have assigned to the commonly used variable `X` for the set of feature variables and `y` for the target variable. It is worthwhile to look at the type of objects assigned to these variables and their dimensions.

In [ ]:
type(X) # X is a numpy multi-dimensional array

In [ ]:
X.shape # Shape attribute shows 200 observations and 5 variables

In [ ]:
X.dtype # Values are 64-bit floating point values (decimals)

In [ ]:
type(y) # y is also a multi-dimensional array

In [ ]:
y.shape # y contains 200 observations

In [ ]:
y.dtype # Two classes encoded as the 64 bit integers 0 and 1

We can use this random data to build a dataframe object.

In [ ]:
df = DataFrame(
    X,
    columns=['Feature1', 'Feature2', 'Feature3', 'Feature4', 'Feature5']
)

df['Target'] = y # Adding target variable

df['TargetClass'] = Series(
    y,
    dtype='category'
) # Adding the target variable again, but specifying it to be categorical

df[:5] # First 5 rows

The `describe` method shows the summary statistics for the variables in the dataframe object.

In [ ]:
df.loc[:, df.columns != 'Target'].describe() # Exclude the Target variable from the summary

A scatter plot matrix shows us the correlation between each set of feature variables for each of the two classes.

In [ ]:
px.scatter_matrix(
    df,
    dimensions=['Feature1', 'Feature2', 'Feature3', 'Feature4', 'Feature5'],
    color='TargetClass', # The categorical version of the target variable
    title='Scatter plot matrix'
)

To use the $k$ nearest neighbour classifier, we instantiate it and specify a value for $k$. We choose $k=5$. There are more arguments available for this classifier, but we leave these at their default values.

In [ ]:
neigh = KNeighborsClassifier(
    n_neighbors=5
)

Next up we fit the data to the instantiated classifier, using the `fit` method.

In [ ]:
neigh.fit(
    X, # The multi-dimensional numpy array of feature variable valuess
    y # The numpy array of target variable values
)

The `predict` method allows us to pass values for an unknown observation and see which class the fitted classifier predicts.

In [ ]:
np.random.seed(7)
unkown_obs = np.random.randn(5).reshape(1, -1) # A single observation with 5 random values

neigh.predict(unkown_obs)

We see a predicted target class of `1`. The `predict_proba` method will return the probability for each target class given an observation.

In [ ]:
neigh.predict_proba(unkown_obs)

Class `1` was predicted with a $100$% probability.

### DATA SPLITTING

While we have _trained_ a classifier, we are not sure how well it does. In most ML applications, we randomly split the dataset into a __training set__ and a __test set__. The model trains of the former (as we did above). The latter is not used in the training, but is kept for obtaining metrics on our model.

This approach allows us to gauge how well a ML model might do on unseen data. This is of obvious importance, as we want to use our model on new data and have it perform well.

This brings with it the concepts of variance and bias, pertaining to how well the data does on the trainig set and how well it performs on unseen data.

High __variance__ refers to a model that does very well on a training set. Such a model __overfits__ the training data and might very well do poorly on unseen data. A model with high __bias__ does rather worse on the training data. To some extent there is a trade-off between these.

A variety of factors influences variance and bias. The sample size is key. The more training data we have in ML, the better the model usually performs. Aspects such as the value of $k$ in our example is termed a __hyperparameter__ of the model. It is _something_ about the model that we choose and must set. Note that there are techniques that can be used to let our computer _search for_ the best hyperparameter values.

Below, we split the data into a $80$% training set and a remainder of $20$% test set. There is a trade-off here too. More data in the test set gives us a better indication of how well it will do on unseen data. We do then, however, take away observations that could have been used for training.

The `train_test_split` function takes various arguments. Below, we set the required arguments. This includes `test_size` set as a fraction of all the observations. We use the commonly used computer variables for the split data. The names are rather explanatory.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

It is important to know that we have a fair representation of the classes in both sets. If not, we have __unbalanced__ sets. This is a particulary interesting problem requiring its own solutions. The numpy `unique` function returns the sample space elements in an array. With the `return_counts` argument set to `True`, we also get a frequency count of each class.

In [ ]:
np.unique(
    y_train,
    return_counts=True
)

In [ ]:
np.unique(
    y_test,
    return_counts=True
)

There is a fair representation of each class in both the training and the test sets.

Now, we train the classifier again (still with $k=5$).

In [ ]:
neigh = KNeighborsClassifier(
    n_neighbors=5
) # Instantiate with k=5

neigh.fit(
    X_train,
    y_train
) # Train on the training data

### METRICS

Given our trained model, we can now pass the unseen test set of feature variables to the model. The predicted target classes are assigned to the computer variable `y_pred` below.

In [ ]:
y_pred = neigh.predict(X_test)

We can now use the predicted target classes to check on various metrics. One important metric is the accuracy. It returns the fraction of values that were precidicted correctly. We use the `accuracy_score` function from the metrics module of the scikit-learn package. As arguments, we pass the actual test target values, `y_test`, and the predicted classes for each test observation, `y_pred`

In [ ]:
metrics.accuracy_score(
    y_test,
    y_pred
)

Our model is $90$% accurate on the unseen data.

A __confusion matrix__ expresses the accuracy by showing the correctly and incorrectly predicted instances. The information in a confusion matix is clear to understand when plotted. We do this with the `plot_confusion_matrix` function from the metrics module of the scickit-learn package.

In [ ]:
metrics.plot_confusion_matrix(neigh, X_test, y_test);

We see the true class labels along the left edge and the predicted class labels on the bottom edge. Looking at the plot, $17$ class `0` observations in the test set were correctly predicted by the model as class `0`, with $19$ class `1` observations correctly predicted. Three actual class `1` observations were incorrectly predicted to be class `0` and a single actual class `0` case was incorrectly prected to be class `1`.

What if we changed the $k$ hyperparameter to be $3$?

In [ ]:
neigh = KNeighborsClassifier(
    n_neighbors=3
) # Instantiate with k=3
neigh.fit(
    X_train,
    y_train
) # Train on the training data

The confusion matrix plot shows that we did a bit better.

In [ ]:
metrics.plot_confusion_matrix(neigh, X_test, y_test);

The accuracy is now up to $92.5$%.

In [ ]:
y_pred = neigh.predict(X_test)

metrics.accuracy_score(
    y_test,
    y_pred
)

## $k$ NEAREST NEIGHBOURS CLASSIFIER DATA SCIENCE EXAMPLE

### DATA IMPORT

In this example we take a data set that can be downloaded from the internet. It contains observations for variables pertaining to the microscopic investigation of cells from breast lumps. Some of the observations are benign (non-canceorus) and some are malignant (cancerous). The spreadsheet file is contained in the `data` subfolder on this Google Drive.

In [ ]:
#drive.flush_and_unmount()

In [ ]:
# Connect to Google Drive
drive.mount('/gdrive', force_remount=True)

# Change directory to the DATA folder
%cd '/gdrive/My Drive/DATA SCIENCE/DATA'

In [ ]:
# Import the spreadsheet file
df = read_csv('breast_cancer.csv')

In [ ]:
# First five observations
df[:5]

The `info` method gives us information about the dataframe object and the variable data types. We note that `diagnosis` is an object data type (a categorical variable).

In [ ]:
# Information about the DataFrame object
df.info()

We can delete the `id` and the `Unnamed: 32` columns as it serves no purpose.

In [ ]:
df.drop(
    ['id', 'Unnamed: 32'],
    axis=1,inplace=True
)

### DATA SUMMARY

There is a known class imbalance in this dataset, with more benign disease than malignant disease. We can visualize and enumerate this.

In [ ]:
px.bar(
    df,
    x='diagnosis',
    title='Frequency of the diagnosis classes',
    labels={
        'diagnosis':'Diagnosis (M for malignant and B for benign)'
    }
)

The fraction of each target class can be calculated using the `value_counts` *method* and setting the `normalize` argument to `True`.

In [ ]:
df.diagnosis.value_counts(normalize=True)

The `describe` method is used to give a summary of the rest of the variables. The `loc` indexing is used to exclude the categorical target variable.

In [ ]:
np.round(
    df.loc[:,df.columns!='diagnosis'].describe(),
    1
) # Rounding to a single secimal place

All the feature variables are numerical variables (for the kNN classifier). We can generate a correlation matrix to investigate the correlation between all pairs of feature variables, using the `corr` method.

In [ ]:
correlation = df.corr()
np.round(correlation, 2) # Rounding to two decimal places

We can visualize these correlations with a heatmap using the matplotlib package.

In [ ]:
plt.figure(
    figsize=(21, 9)
)
plt.title('Correlation of features')
ax = sns.heatmap(
    correlation,
    vmin=-1,
    vmax=1,
    center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    annot=True,
    fmt='.2f',
    linecolor='white'
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=90
)           
plt.show();

### DATA SPLITTING

Before we split the data into a training and a test set, we need to separate the features variables from the target variable.

In [ ]:
# Generate the computer variable X from df with removal of the diagnosis column
y = df.diagnosis # The target variable

X = df.drop(
    ['diagnosis'],
     axis=1
)

We use the `train_test_split` function again to split $20$% of the data as a test set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=12
)

We review the dimensions of the training and test sets.

In [ ]:
X_train.shape, X_test.shape

In [ ]:
y_train.shape, y_test.shape

### DATA SCALING

Scaling data is an important step in ML. It puts all the variables within a similar numerical interval. This has advantages for the training step of many ML algorithms. There are various ways to scale data. Here, we will use __standard scaling__, where the mean of a variable is subtracted from each value in that variable and this difference individed by the standard deviation of that variable, shown in (3), where $z$ is the scaled value, $x_{i}$ is each value for the variable, $\bar{x}$ is the mean and $s_{X}$ the standard deviation of the variable.

$$z = \frac{x_{i} - \bar{X}}{s_{X}} \tag{3}$$

To use this scaler, we instantiate the class.

In [ ]:
# Generating an instance of the StandardScaler class with default argument values
scaler = StandardScaler(
    copy=True,
    with_mean=True,
    with_std=True
)

The training set is first fitted and then transformed (in one step) to the scaler using the `fit_transform` method.

In [ ]:
X_train = scaler.fit_transform(X_train)

The test data is transformed with the attributes of the scaling of the training set. This is very important. The test set must not be scaled using its own mean and standard deviation.

In [ ]:
X_test = scaler.transform(X_test)

### TRAINING

We follow the same steps as used in our initial introduction to the kNN classifier. We will use $k=3$ as hyperparameter value.

In [ ]:
# Instantiating the classifier with k=3
knn = KNeighborsClassifier(
    n_neighbors=3
)

In [ ]:
# Fit the training set
knn.fit(
    X_train,
    y_train
)

### METRICS

The confusion matrix plot shows how well the model faired when using the unseen test data.

In [ ]:
metrics.plot_confusion_matrix(
    knn,
    X_test,
    y_test
);

The accuracy is calculated below using the `accuracy_score` function.

In [ ]:
metrics.accuracy_score(
    y_test,
    knn.predict(X_test)
)

We compare this to the accuracy of the model using the training set, this time using the alternative approach of the `score` method of the model.

In [ ]:
knn.score(
    X_train,
    y_train
)

This gives the same result as the `accuracy_score` function.

In [ ]:
metrics.accuracy_score(
    y_train,
    knn.predict(X_train)
)

There is definitely some overfitting (high variance) as the model does better on the training data than on the test data.

We can also look at the balanced accuracy score using the `balanced_accuracy_score` function. This metric allows for class imbalance.

In [ ]:
metrics.balanced_accuracy_score(
    y_test,
    knn.predict(X_test)
)

This score is still much better than the __null score__, which is the fraction of the majority class. Since `y_test` is a pandas series object, we can use the `value_counts` method with the `normalize` argument set to `True`.

In [ ]:
# Null or baseline score based on proportion majority class
y_test.value_counts(
    normalize=True
)

The majority class in the test set is `B` (benign disease), with a fraction of $0.58$. If we simply use the majority class as predictor, we would be correct $58$% of the time. Our model therefor improves our prediction.

There are other more important metrics such as the sensitivity (recall), the specificity, the positive predictive value (precision), and the negative predictive value. All of these required us to understand the concepts of true and false positive and negatives.

The two classes in our target variable for the current example is `B` for benign and `M` for malignant. As data scientists, we choose one of these classes as our _class of interest_, i.e. the one that we want to predict. In this case, it can be `M`. Given an unknown new observation (set of values for all the feature variables), the model will predict `M` with some probability. We can set a threshold on the interval $\left[ 0,1 \right]$. If the probability for `M` is above the threshold, the model predicts `M`, else it predicts `B`. We can set this threshold depending on how costly mistakes (in either direction) are given the cicumstances in which the model is used. By default, the threshold is $0.5$.

If we look at the first observation in the test set, we note that its true class was `M`.

In [ ]:
y_test.iloc[0]

We can use the first row in the feature set to see what the model predicts.

In [ ]:
knn.predict(X_test[0].reshape(1, -1))

So, if our class of interest was `M`, then the model's prediction would be termed a __true positive__. Here positive refers to the class of interest. If the predicted class was `B`, this would be a __false negative__. Here negative is assigned purely on our research approach and which class we are interested in.

If the true class was `B` and the model predicted a `B`, then this would be a __true negative__. If it predicted `M`, though, it would a `false positive`.

The following abbreviations are often used. The values from our last confusion matrix plot are added under the assumption of `M` being our positive class.

| Metric           | Abbreviation    | Example value |
| :----------------|:----------------|:--------------|
| True positive    | TP              | 42            |
| True negative    | TN              | 66            |
| False positive   | FP              | 0             |
| False negative   | FN              | 6             |


The equations for our four new metrics are shown in (4), where PPV is positive predictive value and NPV is negative predictive value.

$$ \begin{align} \text{sentitivity (recall)} &= \frac{TP}{TP + FN} \\ \text{specificity} &= \frac{TN}{TN + FP} \\ \text{PPV (precision)} &= \frac{TP}{TP + FP} \\ \text{NPV} &= \frac{TN}{TN + FN} \end{align} \tag{4}$$

__Recall__ is then the fraction of true positive cases predicted as such by the model. __Specificity__ is the fraction of true negative cases predicted as such by the model. __Precision__ is the fraction of cases that were correctly predicted as positive over all the cases that were predicted as positive. __Negative predictive value__ is the fraction of cases that were truely negative over all the cases that were predicted to be negative.

>These metrics are domain specific. In healthcare for instance, the sensitivity (more often used in this setting than the Data Science term recall) is the ability of a test to return a positive result given all truely positive cases. Sensitivity is the ability of a test to correctly identity truely negative cases. Positive predictive value (more often used in this setting than the Data Science term precision) is used after the test returns a positive result and expresses the probability of the actual result being positive. Finally, negative predictive value is also used after a test is done and expresses the probability of a negative result actually being negative.

Another metric is the __f1 score__. This reflects the _balance_ between the precision and recall, as shown in (5).

$$f_{1} = 2 \times \frac{\text{precision} \times \text{recall}}{\text{precision} + \text{recall}} = \frac{\text{TP}}{TP + \frac{1}{2} \left(FP + FN \right)} \tag{5}$$

Some of the metrics are returned using the `classification_report` function.

In [ ]:
print(metrics.classification_report(
    y_test,
    knn.predict(X_test)
))

We have raised the question as to when a value is predicted as a certain class. The kNN algorithm produces a probability for each class (the fraction of actual $k$ classes in the neighbourhood of an observation. Since we are dealing with an odd number of neigbours, the majority class in this neigbourhood _rules_. Below, we look at the first $10$ probabilities predicted from the test set feature variables.

In [ ]:
knn.predict_proba(X_test)[0:10]

For most cases all three nearest neighbours were of the same class, but in three of them only two of the neighbours were of the same class.

Given larger values of $k$ we will see different probabilities. Below, we choose $k=7$ and look at the accuracy metric and at the first $10$ observation probabilities again.

In [ ]:
# Instantiate the classifier
knn_7 = KNeighborsClassifier(
    n_neighbors=7
)

In [ ]:
# Train the model
knn_7.fit(
    X_train,
    y_train
)

In [ ]:
# Accuracy on the test set
knn_7.score(
    X_test,
    y_test
)

We get an improved accuracy.

In [ ]:
knn_7.predict_proba(X_test)[0:10]

Below, we generate a DataFrame object from these probabilities (using the test set).

In [ ]:
probabilities = DataFrame(
    knn_7.predict_proba(X_test),
    columns=['B', 'M']
)

probabilities[:10]

We can now create a bar chart to show the frequency of each probability for the `M` class.

In [ ]:
px.bar(
    np.round(probabilities, 2),
    x='M',
    title='Frequency of probabilities for malignant class',
    labels={'M':'Probabilities of M'}
).update_xaxes(
    type='category'
)

All the observations that had a probability of `M` greater than $50$% ($0.5$) is predicted to be `M` by the model. What if we change this threshold, though? This can be done depending on how _expensive_ mistakes are. If it is costly to miss a positive results, then we can set the threshold lower so that more observations are predicted to be positive. The meaning of the term _expensive_ is determined by the setting of the Data Science project.

A __receiver operator characteristic__ (ROC) curve presents a visual representation of different thresholds. The _x_ axis of this plot is $1 - \text{specificity}$ and the _y_ axis is the recall. To use this plot, we first calculate the required values using the `roc_curve` function.

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(
    y_test.replace(['B', 'M'], [0, 1]),
    knn_7.predict_proba(
        X_test
    )[:, 1]
)

Below, we use the matplotlib package to generate the curve.

In [ ]:
plt.plot(fpr, tpr, linewidth=2)
plt.plot([0,1], [0,1], 'o--')
plt.rcParams['font.size'] = 12
plt.title('ROC curve kNN classifier')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')

plt.show();

The orange dotted line represents a $50:50$ _chance_. We want our curve to be higher than this line, which indeed it is. For a specificity of just under $100$% (to the left of the _x_ axis), we get almost $90$% recall.

The area under the curve(ROC AUC) represents the ROC score. The closer the ROC AUC is to $1.0$ the better the model performance. The `roc_auc_score` function calucates this area. Note that we use the numpy `where` function to replace `B` with $0$ and `M` with $1$ since we need numerical values.

In [ ]:
metrics.roc_auc_score(
    y_test.replace(['B', 'M'], [0, 1]),
    np.where(
        knn_7.predict(X_test) == 'B', 0, 1
    )
)

The ROC score or ROC AUC is $0.95$, which is very good.

For both kNN classifiers ($k=3$ and $k=7$) we have only performed a single training step. We might have been very _lucky_ or _unlucky_ in the random split of a training and a test set. It is better to repeat this process many times over. This is termed cross-validation.

### CROSS VALIDATION

With __cross-validation__ we split the data repeatedly and measure performance metrics, over which we average in the end. In $k$ fold cross validation, we choose a number of folds. As an example, we might choose $k=5$. Note that this is not the $k$ of kNN. For a $5$ fold cross validation, the data is split into training and test sets five times, such that all the data is used for training and testing.

When the `scoring` argument is set to `accuracy`, the `cross_val_score` function from the model_selection module of the scikit-learn package returns the accuracy $k$ number of times.

In [ ]:
scores = cross_val_score(
    knn_7,
    X,
    y,
    cv=5,
    scoring='accuracy'
)

In [ ]:
scores

The average of these scores gives a better understanding of model performance on unseen data.

In [ ]:
np.mean(scores)

In [ ]:
np.min(scores), np.max(scores)

There is quite a large range for these scores, indicating that the accuracy is very dependent on which observations are in the training and the test sets. In this case, it is a function of the small number of observations in the data set.

The final question in this section is which value of $k$ for the kNN classifier is best. One method for finding the optimal value is to perform a grid search.

### GRID SEARCH FOR BEST HYPERPARAMETER VALUES

With a grid search we can explore the solution space for hyperparameter values. This process is known as __hyperparameter tuning__.

The hyperparameters we will tune for the kNN classifier here are the leaf count, the number of neighbours, and the distance metric. While we used Euclidean distance in this notebook, there are other distance metrics too. This argument is set using the `p` value when instatiating the classifier. The leaf count pertains to the search algorithm used for determining the closest neighbours. The kNN classifier used in scikit-learn can use a few of these algorithms such as the KD-tree algorithm or the ball-tree algorithm, or even a brute force approach.

To use a grid search, we set values to explore.

In [ ]:
# Generate a Python list of leaf size values on the closed interval 1 through 49
leaf_size = list(range(1, 50))

# Generate a Python list of neigbour numbers on the closed interval 1 through 19
n_neighbors = list(range(1, 20))

p = [1, 2]

We convert these lists to a dictionary.

In [ ]:
hyperparams = {
    'leaf_size':leaf_size,
    'n_neighbors':n_neighbors,
    'p':p
}

Next we instantiate a new kNN classifier with default argument values.

In [ ]:
knn = KNeighborsClassifier()

Now we perform the grid search, which will go through all the hyperparameter values when we fit the data. We also use $5$ fold cross validation. All of this can be computationally expensive (consuming a lot a computer resources and taking a long time). 

In [ ]:
knn_grid_search = GridSearchCV(
    knn,
    hyperparams,
    cv=5
)

In [ ]:
best_params = knn_grid_search.fit(
    X,
    y
)

The process took over 90 seconds on Colab.

Now we can print the best hyperparameter values using the `best_estimator.get_params` method.

In [ ]:
# Best leaf size
best_params.best_estimator_.get_params()['leaf_size']

In [ ]:
# Best number of neighbours
best_params.best_estimator_.get_params()['n_neighbors']

In [ ]:
# Best distance metric
best_params.best_estimator_.get_params()['p']

Below, we use these hyperparameter values and $5$ fold cross validation. Note that these may be different every time you run the code. Below, we see the best parameter values generated during a previous run.

In [ ]:
knn_best = KNeighborsClassifier(
    n_neighbors=9,
    leaf_size=9,
    p=1
)

In [ ]:
scores = cross_val_score(
    knn_best,
    X,
    y,
    cv=5,
    scoring='accuracy'
)

The average accuracy is now better than before.

In [ ]:
np.mean(scores)

## $k$ NEAREST NEIGHBOUR REGRESSION

The $k$ nearest neighbour (kNN) algorithm can also be used for regression. Here the target variable is a continuous numerical variable.

### GENERATING A DATA SET

To understand the basic concept of building a kNN regression model, we start by generating a data set, with a single feaure variable, and then visualise the data.

In [ ]:
X = np.arange(
    start=1,
    stop=11
)

X # A 10 element array

In [ ]:
y = 2 * X

y

In [ ]:
go.Figure(
    go.Scatter(
        x=X,
        y=y,
        name='Data',
        mode='markers',
        marker={
            'size':20
        }
    )
).update_yaxes(
    scaleanchor='x',
    scaleratio=1
).update_layout(
    title='Regression data',
    xaxis={'title':'Feature variable'},
    yaxis={'title':'Target variable'}
)

### CREATING A MODEL

We instantiate a kNN regressor with $k=3$ nearest neighbours. The leaf size, search method, and distance measure arguments are left at their default values.

In [ ]:
# Instantiating a kNN regressor with k=3 neighbours
knn = KNeighborsRegressor(
    n_neighbors=3
)

### TRAINING THE MODEL

Next, we fit the data to the model for training.

In [ ]:
knn.fit(
    X.reshape(-1, 1),
    y
)

### TESTING THE MODEL

We can now pass a value to the model to see what it predicts and then try to understand the result.

In [ ]:
knn.predict(np.array([5.5]).reshape(1, -1))

We see a result of $10$. We can plot this to visualize the prediction in view of the data.

In [ ]:
go.Figure(
    go.Scatter(
        x=X,
        y=y,
        name='Data',
        mode='markers',
        marker={
            'size':20
        }
    )
).add_trace(
    go.Scatter(
        x=[5.5],
        y=[10],
        name='Unseen data',
        mode='markers',
        marker={
            'size':20
        }
    )
).update_yaxes(
    scaleanchor='x',
    scaleratio=1
).update_layout(
    title='Regression data',
    xaxis={'title':'Feature variable'},
    yaxis={'title':'Target variable'}
)

The three nearest observations have target variable values of $9, 10$, and $11$. The average of this is $10$.

## CONCLUSION

This notebook was an introduction to the world of machine learning using one of the most interpretable algorithms. Through the examples, we have gained valueble knowledge of terms used in machine learning, the construction of these models, and how to evaluate them.